In [1]:
from cryptography.hazmat.backends import default_backend

In [2]:
from cryptography.hazmat.primitives import serialization

In [4]:
pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=21d788cdf40cc2a3eb6a61c751377b2bb31014b03f4a7661388b1b47ec9f6e09
  Stored in directory: c:\users\lohit\appdata\local\pip\cache\wheels\27\3e\a7\888155c6a7f230b13a394f4999b90fdfaed00596c68d3de307
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyspark import SparkConf,SparkContext

In [8]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.context import SparkContext
import re
import sys
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.master("yarn")\
                .appName('TerraLoad') \
                .getOrCreate()


We are trying to connect to snowflake and fetch the data available in snowflake. 

We have many authentication techniques but we are using private/public key authentication. 

before trying to authenticate here, we need to add public key to snowflake as authorized user(ssh_public_key). 

This below code is to take the private key file and password, deserialize it for further for consumption




In [ ]:
with open("path_to_rsa_key.p8","rb") as key_file:
    p_key = serialization.load_pem_private_key(
    key_file.read(),
    password="private key file password".encode(),
    backend=default_backend())

In [ ]:
pkb = p_key.private_bytes(encoding = serialization.Encoding.PEM,format=serialization.PrivateFormat)

In [ ]:

pkb = p_key.private_bytes(encoding-serialization.Encoding.PEM,format=serialization.PrivateFormat.PKCS8, 
                          encryption_algorithm=serialization.NoEncryption())

pkb = pkb.decode("UTF-8")

pkb = re.sub("-*(BEGIN|END) PRIVATE KEY-*\n", "" ,pkb). replace("\n", "")

sfOptions ={
"sfURL" : "https://<account>.east-us-2.azure.snowflakecomputing.com",
"sfAccount" : "<account>",
"'sfUser" :"lohithkumar036@gmail.com"
"sfDatabase": "PROSPECTIVE_DB",
"sfSchema" : "FOUNDATION",
"sfWarehouse" : "PROSPECTIVE_ANALYTICS_WH",
"sfRole" : "AR_ROLE_LOHITHKUMAR_ROLE",
"pem_private_key":pkb
}

snowflake_source_name = "net.snowflake.spark.snowflake"
df = spark.read.format(snowflake_source_name)\
        .options(**sfOptions)\
        .option("query","""
        select * from table <your query here>
        """).load()

    
# if you want to save data as parquet file in hdfs location 
df.repartition(1).write.mode ("overwrite").parquet("<location where you wanted to save as parquet>")

#if you want to save data directly to terradata
df.write. format("jdbc")\
    .option ("driver", "com.teradata.jdbc.TeraDriver") \
    .option ("url","jdbc:teradata://<account>/DATABASE=***,DBS_PORT=***, LOGMECH=LDAP, TYPE=FASTEXPORT,TMODE=TERA")\
    .option ("dbtable", "<table name>")\ 
    .option ("user", "<username>")\ 
    .option ("password", "<password>")\ 
    .mode ("append" )\
    .save ()